In [2]:
import pandas as pd
import numpy as np

from catboost import CatBoostRegressor
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

from datetime import datetime

pd.set_option('display.max_rows', 5000)

In [3]:
zhi_df=pd.read_csv('../data/Zip_Zri_AllHomesPlusMultifamily.csv')


In [4]:
zhi_df2=pd.read_csv('../data/Zip_Zri_MultiFamilyResidenceRental.csv')


In [5]:
zhi_df.shape

(13181, 120)

In [6]:
zhi_df2.shape

(1861, 120)

In [7]:
date_mapping={i:j for i,j in enumerate(zhi_df.columns[6:])}

In [47]:
#zri = pd.melt(zhi_df, id_vars = zhi_df.columns[0:6], value_vars = zhi_df.columns[7:], var_name = 'year-month', value_name = 'ZRI').sort_values(['RegionID', 'year-month'])

zri = pd.melt(zhi_df2, id_vars = zhi_df2.columns[0:6], value_vars = zhi_df2.columns[7:], var_name = 'year-month', value_name = 'ZRI').sort_values(['RegionID', 'year-month'])


zri=zri[(zri['year-month']>'2013-07') & (zri['year-month']<'2020-01')]

zri=zri.fillna(0)
zri['gapsize']=zri['ZRI'].rolling(5).sum()
droplist=list(zri[((zri['gapsize']==0) | zri['gapsize'].isna()) & (zri['year-month']>'2014-02')]['RegionID'].drop_duplicates())
zri[(zri['gapsize']==0)].shape[0]/zri.shape[0]
len(droplist)/len(zri['RegionID'].drop_duplicates())



0.3197205803331542

In [51]:
zri2 = pd.melt(zhi_df2, id_vars = zhi_df2.columns[0:6], value_vars = zhi_df2.columns[7:], var_name = 'year-month', value_name = 'ZRI').sort_values(['RegionID', 'year-month'])

zri2=zri2.groupby('RegionID').apply(lambda group: group.interpolate())
#zri2[zri2['ZRI'].isna()]

#zri2['ZRI']=(zri2['ZRI']/zri2['ZRI'].shift(1))-1

zri2=zri2[(zri2['year-month']>'2014-02') & (zri2['year-month']<'2020-01')]

zri3=zri2[~(zri2['RegionID'].isin(droplist))]


In [54]:
len(zri3['RegionID'].drop_duplicates())

1266

In [53]:
zri3.isna().sum()

RegionID       0
RegionName     0
City           0
State          0
Metro         70
CountyName     0
year-month     0
ZRI            0
dtype: int64

In [9]:
def laggenerator(i,colname,df):
    timelist=list(df['year-month'].drop_duplicates().sort_values())[0:i]
    df.loc[:,f'{colname}_lag{i}']=df.loc[:,f'{colname}'].shift(i)
    df.loc[df['year-month'].isin(timelist),f'{colname}_lag{i}']=0
    return df

In [10]:
zri3=laggenerator(1, 'ZRI',zri3)
zri3=laggenerator(2, 'ZRI',zri3)
zri3=laggenerator(3, 'ZRI',zri3)

/Users/moritz/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/Users/moritz/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/Users/moritz/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.

In [11]:
train, test=zri3[(zri3['year-month']<='2018-12') & (zri3['year-month']>='2014-01')], zri3[(zri3['year-month']>'2018-12') & (zri3['year-month']<='2019-12')]

train=train.drop('year-month',axis=1)

Y_train, Y_test=train['ZRI'], test[['ZRI']]
X_train, X_test=train.drop('ZRI',axis=1), test.drop('ZRI',axis=1)



In [12]:
labelencoder = LabelEncoder()
X_train['CountyNameEnc'] = labelencoder.fit_transform(X_train['CountyName'])
X_train=X_train.drop(['CountyName','RegionID','RegionName','City','State','Metro'],axis=1)


In [13]:
cat=CatBoostRegressor()

In [14]:
cat.fit(X_train, Y_train)

Learning rate set to 0.123847
0:	learn: 0.0092331	total: 89.6ms	remaining: 1m 29s
1:	learn: 0.0088771	total: 111ms	remaining: 55.5s
2:	learn: 0.0085812	total: 133ms	remaining: 44.4s
3:	learn: 0.0083441	total: 160ms	remaining: 39.8s
4:	learn: 0.0081510	total: 185ms	remaining: 36.9s
5:	learn: 0.0079918	total: 210ms	remaining: 34.8s
6:	learn: 0.0078650	total: 235ms	remaining: 33.3s
7:	learn: 0.0077587	total: 258ms	remaining: 32s
8:	learn: 0.0076728	total: 280ms	remaining: 30.8s
9:	learn: 0.0076027	total: 302ms	remaining: 29.9s
10:	learn: 0.0075463	total: 325ms	remaining: 29.2s
11:	learn: 0.0075004	total: 347ms	remaining: 28.6s
12:	learn: 0.0074633	total: 371ms	remaining: 28.2s
13:	learn: 0.0074335	total: 396ms	remaining: 27.9s
14:	learn: 0.0074087	total: 417ms	remaining: 27.4s
15:	learn: 0.0073886	total: 441ms	remaining: 27.1s
16:	learn: 0.0073721	total: 463ms	remaining: 26.8s
17:	learn: 0.0073569	total: 483ms	remaining: 26.4s
18:	learn: 0.0073452	total: 505ms	remaining: 26.1s
19:	learn: 

167:	learn: 0.0072364	total: 3.82s	remaining: 18.9s
168:	learn: 0.0072362	total: 3.84s	remaining: 18.9s
169:	learn: 0.0072361	total: 3.86s	remaining: 18.9s
170:	learn: 0.0072358	total: 3.89s	remaining: 18.8s
171:	learn: 0.0072357	total: 3.91s	remaining: 18.8s
172:	learn: 0.0072355	total: 3.94s	remaining: 18.8s
173:	learn: 0.0072354	total: 3.96s	remaining: 18.8s
174:	learn: 0.0072352	total: 3.98s	remaining: 18.8s
175:	learn: 0.0072350	total: 4s	remaining: 18.7s
176:	learn: 0.0072346	total: 4.03s	remaining: 18.7s
177:	learn: 0.0072342	total: 4.05s	remaining: 18.7s
178:	learn: 0.0072341	total: 4.08s	remaining: 18.7s
179:	learn: 0.0072339	total: 4.1s	remaining: 18.7s
180:	learn: 0.0072338	total: 4.12s	remaining: 18.7s
181:	learn: 0.0072336	total: 4.14s	remaining: 18.6s
182:	learn: 0.0072335	total: 4.16s	remaining: 18.6s
183:	learn: 0.0072333	total: 4.18s	remaining: 18.6s
184:	learn: 0.0072331	total: 4.21s	remaining: 18.5s
185:	learn: 0.0072330	total: 4.23s	remaining: 18.5s
186:	learn: 0.00

328:	learn: 0.0072111	total: 7.25s	remaining: 14.8s
329:	learn: 0.0072110	total: 7.26s	remaining: 14.7s
330:	learn: 0.0072108	total: 7.29s	remaining: 14.7s
331:	learn: 0.0072106	total: 7.31s	remaining: 14.7s
332:	learn: 0.0072105	total: 7.33s	remaining: 14.7s
333:	learn: 0.0072104	total: 7.35s	remaining: 14.7s
334:	learn: 0.0072103	total: 7.37s	remaining: 14.6s
335:	learn: 0.0072102	total: 7.39s	remaining: 14.6s
336:	learn: 0.0072100	total: 7.41s	remaining: 14.6s
337:	learn: 0.0072099	total: 7.42s	remaining: 14.5s
338:	learn: 0.0072098	total: 7.44s	remaining: 14.5s
339:	learn: 0.0072096	total: 7.46s	remaining: 14.5s
340:	learn: 0.0072095	total: 7.48s	remaining: 14.5s
341:	learn: 0.0072094	total: 7.5s	remaining: 14.4s
342:	learn: 0.0072093	total: 7.52s	remaining: 14.4s
343:	learn: 0.0072092	total: 7.54s	remaining: 14.4s
344:	learn: 0.0072091	total: 7.55s	remaining: 14.3s
345:	learn: 0.0072090	total: 7.57s	remaining: 14.3s
346:	learn: 0.0072088	total: 7.59s	remaining: 14.3s
347:	learn: 0

496:	learn: 0.0071920	total: 10.6s	remaining: 10.8s
497:	learn: 0.0071919	total: 10.7s	remaining: 10.7s
498:	learn: 0.0071918	total: 10.7s	remaining: 10.7s
499:	learn: 0.0071917	total: 10.7s	remaining: 10.7s
500:	learn: 0.0071916	total: 10.7s	remaining: 10.7s
501:	learn: 0.0071915	total: 10.7s	remaining: 10.7s
502:	learn: 0.0071914	total: 10.8s	remaining: 10.6s
503:	learn: 0.0071913	total: 10.8s	remaining: 10.6s
504:	learn: 0.0071911	total: 10.8s	remaining: 10.6s
505:	learn: 0.0071911	total: 10.8s	remaining: 10.6s
506:	learn: 0.0071909	total: 10.8s	remaining: 10.5s
507:	learn: 0.0071908	total: 10.9s	remaining: 10.5s
508:	learn: 0.0071907	total: 10.9s	remaining: 10.5s
509:	learn: 0.0071905	total: 10.9s	remaining: 10.5s
510:	learn: 0.0071904	total: 10.9s	remaining: 10.4s
511:	learn: 0.0071902	total: 10.9s	remaining: 10.4s
512:	learn: 0.0071901	total: 11s	remaining: 10.4s
513:	learn: 0.0071900	total: 11s	remaining: 10.4s
514:	learn: 0.0071899	total: 11s	remaining: 10.4s
515:	learn: 0.0071

664:	learn: 0.0071750	total: 14s	remaining: 7.06s
665:	learn: 0.0071749	total: 14s	remaining: 7.04s
666:	learn: 0.0071748	total: 14.1s	remaining: 7.02s
667:	learn: 0.0071747	total: 14.1s	remaining: 7s
668:	learn: 0.0071745	total: 14.1s	remaining: 6.98s
669:	learn: 0.0071744	total: 14.1s	remaining: 6.95s
670:	learn: 0.0071744	total: 14.1s	remaining: 6.93s
671:	learn: 0.0071743	total: 14.2s	remaining: 6.91s
672:	learn: 0.0071742	total: 14.2s	remaining: 6.89s
673:	learn: 0.0071741	total: 14.2s	remaining: 6.87s
674:	learn: 0.0071740	total: 14.2s	remaining: 6.85s
675:	learn: 0.0071739	total: 14.2s	remaining: 6.83s
676:	learn: 0.0071738	total: 14.3s	remaining: 6.81s
677:	learn: 0.0071737	total: 14.3s	remaining: 6.79s
678:	learn: 0.0071736	total: 14.3s	remaining: 6.76s
679:	learn: 0.0071735	total: 14.3s	remaining: 6.74s
680:	learn: 0.0071734	total: 14.3s	remaining: 6.72s
681:	learn: 0.0071733	total: 14.4s	remaining: 6.7s
682:	learn: 0.0071732	total: 14.4s	remaining: 6.68s
683:	learn: 0.007173

830:	learn: 0.0071590	total: 17.4s	remaining: 3.54s
831:	learn: 0.0071589	total: 17.4s	remaining: 3.52s
832:	learn: 0.0071588	total: 17.4s	remaining: 3.5s
833:	learn: 0.0071588	total: 17.5s	remaining: 3.48s
834:	learn: 0.0071587	total: 17.5s	remaining: 3.46s
835:	learn: 0.0071586	total: 17.5s	remaining: 3.43s
836:	learn: 0.0071585	total: 17.5s	remaining: 3.41s
837:	learn: 0.0071584	total: 17.5s	remaining: 3.39s
838:	learn: 0.0071583	total: 17.6s	remaining: 3.37s
839:	learn: 0.0071582	total: 17.6s	remaining: 3.35s
840:	learn: 0.0071582	total: 17.6s	remaining: 3.33s
841:	learn: 0.0071581	total: 17.6s	remaining: 3.31s
842:	learn: 0.0071579	total: 17.7s	remaining: 3.29s
843:	learn: 0.0071578	total: 17.7s	remaining: 3.27s
844:	learn: 0.0071577	total: 17.7s	remaining: 3.25s
845:	learn: 0.0071577	total: 17.7s	remaining: 3.23s
846:	learn: 0.0071575	total: 17.7s	remaining: 3.2s
847:	learn: 0.0071574	total: 17.8s	remaining: 3.18s
848:	learn: 0.0071573	total: 17.8s	remaining: 3.16s
849:	learn: 0.

997:	learn: 0.0071449	total: 20.8s	remaining: 41.7ms
998:	learn: 0.0071448	total: 20.8s	remaining: 20.8ms
999:	learn: 0.0071447	total: 20.8s	remaining: 0us


In [15]:
#loop months
#for January: filter Jan
#predict Jan
#update all lags
#loop

predictor_table=zri3.copy()

#loop through all month in 2019
for month in list(X_test['year-month'].drop_duplicates()):
    X_test2=X_test.copy()
    
    #run prediction for one month
    X_test2=X_test2[X_test2['year-month']==month]
    X_test2['CountyNameEnc'] = labelencoder.transform(X_test2['CountyName'])
    X_test2=X_test2.drop(['CountyName','RegionID','RegionName','City','State','Metro','year-month'],axis=1)
    val=cat.predict(X_test2)
    
    #define nextmonth
    nextmonth=date_mapping[[mon_ind for mon_ind, name in date_mapping.items() if name == month][0]+1]

    #write current month prediction into predictor_table
    if nextmonth<'2020-01':
        predictor_table.loc[predictor_table['year-month']==nextmonth,'ZRI']=val
        
        predictor_table=laggenerator(1, 'ZRI',predictor_table)
        predictor_table=laggenerator(2, 'ZRI',predictor_table)
        predictor_table=laggenerator(3, 'ZRI',predictor_table)
        
        X_test=predictor_table[predictor_table['year-month']>'2018-12']



In [ ]:
RSME=(sum((Y_test-X_test['ZRI'])**2)/len(Y_test))**0.5
RSME

In [ ]:
Y_test.mean()

In [ ]:
X_test

In [ ]:
Y_test

In [ ]:
#enc = OneHotEncoder(handle_unknown='ignore')

#X_train_enc = pd.DataFrame(enc.fit_transform(X_train[['CountyName']]).toarray())

#X_train = X_train.join(X_train_enc)
#X_train

#,

In [ ]:
testzip=zri3[zri3['RegionID']==753844]
testzip['year-month']=pd.to_datetime(testzip['year-month'])
testzip=testzip.set_index('year-month')
from statsmodels.tsa.seasonal import seasonal_decompose
import plotly.express as px

decomposed=seasonal_decompose(testzip['ZRI'],model='additive')

trend=decomposed.trend
seasonal=decomposed.seasonal
residual=decomposed.resid

#px.line(testzip['ZRI'])
#px.line(trend)
#px.line(seasonal)
#px.line(residual)

In [ ]:
from pmdarima.arima import auto_arima

x=100
X_train, X_test=testzip[0:x], testzip[x:len(testzip)-1]

arima_model=auto_arima(X_train['ZRI'], start_p=1, start_q=1, max_p=5, max_q=5, m=12, start_P=0, start_D=0,
                       start_Q=0, max_P=5, max_D=5, max_Q=5, 
                       seasonal=True, trace=True, stepwise=True, n_fits=50)

In [ ]:
zhi_reshaped2.gapsize==0

In [ ]:
zhi_reshaped2[(zhi_reshaped2['gapsize']==0)]

In [ ]:
zhi_reshaped2.fillna(0)
zhi_reshaped2['gapsize']=zhi_reshaped2['Value'].rolling(1).sum()

In [ ]:
zhi_reshaped2['gapsize']

In [ ]:
zhi_df.isna().sum(axis=1).shape

In [ ]:
zhi_df.isna().sum(axis=1)<1

In [ ]:
zhi_df[(zhi_df.isna().sum(axis=1)>8)].shape

In [ ]:
zhi_reshaped.groupby('RegionID').agg('count')

In [ ]:
zhi_reshaped

In [ ]:
acs_df=pd.read_csv('../data/acs_5yr_merged.csv')

In [ ]:
acs_df.shape

In [ ]:
acs_df.isna().sum()